# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'FilterInterface',
 'FilterModeW',
 'PacketSettingsW',
 'PacketW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface = await new_transport_interface("0.0.0.0", 12345)
tiface.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f3296053510>()]>

# Filter layer setup

In [4]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [7]:
[m for m in dir(fiface) if not m.startswith('__')]

['run']

Run Filter!

In [8]:
await fiface.run()

thread '<unnamed>' panicked at 'there is no reactor running, must be called from the context of a Tokio 1.x runtime', nwv2-python-wrapper/src/filter/mod.rs:113:9
stack backtrace:
   0: rust_begin_unwind
             at /rustc/a55dd71d5fb0ec5a6a3a9e8c27b2127ba491ce52/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/a55dd71d5fb0ec5a6a3a9e8c27b2127ba491ce52/library/core/src/panicking.rs:142:14
   2: core::panicking::panic_display
             at /rustc/a55dd71d5fb0ec5a6a3a9e8c27b2127ba491ce52/library/core/src/panicking.rs:72:5
   3: core::panicking::panic_str
             at /rustc/a55dd71d5fb0ec5a6a3a9e8c27b2127ba491ce52/library/core/src/panicking.rs:56:5
   4: core::option::expect_failed
             at /rustc/a55dd71d5fb0ec5a6a3a9e8c27b2127ba491ce52/library/core/src/option.rs:1874:5
   5: core::option::Option<T>::expect
             at /rustc/a55dd71d5fb0ec5a6a3a9e8c27b2127ba491ce52/library/core/src/option.rs:738:21
   6: tokio::task::spawn::sp

PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime